In [5]:
import pulp
from pprint import pprint
import pandas as pd
import numpy as np

df = pd.read_csv('E:\\private\\学习\\2020-bi-programing-practice\\随便写写\\intelligen_logistics\\L22\\santa\\family_data.csv',index_col='family_id')
df

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


#### 1）计算Perference Cost矩阵 pcost_mat
####  2）计算Accounting Cost矩阵 acost_mat
#### 3）计算每个家庭的人数 FAMILY_SIZE
#### 4）每个家庭的倾向选择（choice_） DESIRED


In [8]:
N_DAYS = 100
N_FAMILY=5000
MAX_OCCUPANCY=300
MIN_OCCUPANCY=125
pcost_mat = np.full(shape=(N_FAMILY, N_DAYS), fill_value= np.inf)

array([[inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       ...,
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf]])

In [57]:
def get_penalty(n_people, i):
    n_choice = df.loc[i,:].values - 1
    temp = np.zeros((N_DAYS))
    for i in range(N_DAYS):
        if i == n_choice[0]:
            temp[i] = 0
        elif i== n_choice[1]:
            temp[i] = 50
        elif i == n_choice[2]:
            temp[i] = 50 + n_people*9
        elif i == n_choice[3]:
            temp[i] = 100 + n_people*9
        elif i==n_choice[4]:
            temp[i] = 200 + n_people*9
        elif i==n_choice[5]:
            temp[i] = 200 + n_people*18
        elif i==n_choice[6]:
            temp[i] = 300 + n_people*18
        elif i == n_choice[7]:
            temp[i] = 300 + n_people*36
        elif i== n_choice[8]:
            temp[i] = 400 + n_people*36
        elif i == n_choice[9]:
            temp[i] = 500 + n_people*(36+199)
        else:
            temp[i] = 500 + n_people*(36+398)
    return temp

In [58]:
%%time
for i in range(N_FAMILY):
    n_people = df.loc[i, 'n_people']
    pcost_mat[i,:] = get_penalty(n_people, i)

Wall time: 3.15 s


In [81]:
acost_mat = np.zeros((MAX_OCCUPANCY, MAX_OCCUPANCY), dtype='float64')

发现安排人数较少时，财务生成也很高，应该是和财务公式相关，如果人数较少其实也是不划算的，所以要将300\*300的矩阵值都计算,165*165的方案不好

In [83]:
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]):
        acost_mat[i,j] = max(0, ((i+1-125)/400)*((i+1)** (1/2+abs(i-j+1)/50)))

In [101]:
FAMILY_SIZE = df['n_people'].values
DESIRED = df.iloc[:,:-1].values -1

#### Step3，使用LP和MIP求解 规划方案
#### 1）先使用LP 对绝大部分家庭进行规划
#### 2）再使用MIP 对剩余家庭进行规划
#### 3）汇总两边的结果 => 最终规划方案


In [217]:
from ortools.linear_solver import pywraplp
#做LP线性规划
def solveLP():
    solver = pywraplp.Solver('santa', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x = {}  #famliyID 在第几天参观
    #每一天都有哪些家庭来参观
    candidates = [[] for i in range(N_DAYS)]
    for i in range(N_FAMILY):
        for j in DESIRED[i,:]:
            candidates[j].append(i)
            #定义变量，x[i,j]第i个家庭，第j天是否参加
            x[i,j] = solver.BoolVar('x[%i,%i]'%(i,j))
    print(len(x))
    #每天人数125-300，统计每天的人数
    daliy_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]]) for j in range(N_DAYS)]
    #家庭在10选择中出现的次数
    famliy_presence = [solver.Sum([x[i,j] for j in DESIRED[i,:]]) for i in range(N_FAMILY)]
    
    #目标函数
    preference_cost = solver.Sum([pcost_mat[i,j] *x[i,j] for i in range(N_FAMILY) for j in DESIRED[i,:]])
    solver.Minimize(preference_cost)
    #设置约束条件
    #当天出现人数不超过前一天25
    for j in range(N_DAYS-1):
        solver.Add(daliy_occupancy[j] - daliy_occupancy[j+1] <=25)
        solver.Add(daliy_occupancy[j+1] - daliy_occupancy[j] <=25)
        
    for i in range(N_FAMILY):
        solver.Add(famliy_presence[i] == 1)
    for j in range(N_DAYS):
        solver.Add(daliy_occupancy[j] >= MIN_OCCUPANCY)
        solver.Add(daliy_occupancy[j] <= MAX_OCCUPANCY)
    result = solver.Solve()
    temp = [(i,j,x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:] if x[i,j].solution_value() >0]
    df = pd.DataFrame(temp, columns=['family_id', 'day','result'])
    return df
    #print(daliy_occupancy)
    #print(candidates)
        

In [218]:
%%time
result = solveLP()
result

50000


,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5068,4995,15,1.0
5069,4996,87,1.0
5070,4997,31,1.0
5071,4998,91,1.0


In [221]:
temp = result['family_id'].value_counts()
print(len(temp[temp > 1]))
result['result'].value_counts()
result[result['family_id'].isin(temp[temp > 1].index)]

73


,family_id,day,result
59,59,38,0.25
60,59,14,0.75
241,240,32,0.75
242,240,56,0.25
264,262,31,0.50
...,...,...,...
4983,4912,8,0.40
4985,4914,38,0.60
4986,4914,43,0.40
5033,4961,53,0.75


线性规划虽然设置参数为bool型，但是从result值中可以看到并不是0和1两个，而是很多小数，类似概率，所以result的family_id有重复